In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.utils import resample
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score

from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler



from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, roc_curve, auc
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn import tree 
from sklearn.preprocessing import LabelEncoder
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics

In [32]:
url = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?$where=created_date%20between%20%272019-04-11T00:00:00%27%20and%20%272019-04-12T00:00:00%27"
data = pd.read_json(url)

In [33]:
data.head()

,address_type,agency,agency_name,bbl,borough,bridge_highway_direction,bridge_highway_name,bridge_highway_segment,city,closed_date,...,park_facility_name,resolution_action_updated_date,resolution_description,road_ramp,status,street_name,taxi_pick_up_location,unique_key,x_coordinate_state_plane,y_coordinate_state_plane
0,ADDRESS,DOHMH,Department of Health and Mental Hygiene,2.026360e+09,BRONX,NaN,NaN,NaN,BRONX,2019-04-17T17:23:06.000,...,Unspecified,2019-04-17T17:23:06.000,The Department of Health and Mental Hygiene wi...,NaN,Closed,JACKSON AVENUE,NaN,42185961,1010035.0,237583.0
1,INTERSECTION,DOHMH,Department of Health and Mental Hygiene,NaN,QUEENS,NaN,NaN,NaN,Far Rockaway,2019-04-17T00:00:00.000,...,Unspecified,2019-04-17T00:00:00.000,The Department of Health and Mental Hygiene wi...,NaN,Closed,NaN,NaN,42192716,1034103.0,158016.0
2,ADDRESS,DOHMH,Department of Health and Mental Hygiene,2.057030e+09,BRONX,NaN,NaN,NaN,BRONX,2019-04-22T00:00:00.000,...,Unspecified,2019-04-22T00:00:00.000,The Department of Health and Mental Hygiene wi...,NaN,Closed,WEST 230 STREET,NaN,42195383,1009847.0,259067.0
3,ADDRESS,DOHMH,Department of Health and Mental Hygiene,NaN,MANHATTAN,NaN,NaN,NaN,NEW YORK,2019-04-11T00:00:01.000,...,Unspecified,2019-04-22T13:05:50.000,Callers should contact the DOHMH Foodborne Ill...,NaN,Closed,EAST 24 STREET,NaN,42195875,987900.0,209335.0
4,ADDRESS,DEP,Department of Environmental Protection,4.019370e+09,QUEENS,NaN,NaN,NaN,Corona,2019-04-14T00:00:00.000,...,Unspecified,2019-04-14T00:00:00.000,The Department of Environmental Protection req...,NaN,Closed,100 STREET,NaN,42197463,1023037.0,208811.0


In [34]:
data.columns

Index(['address_type', 'agency', 'agency_name', 'bbl', 'borough',
       'bridge_highway_direction', 'bridge_highway_name',
       'bridge_highway_segment', 'city', 'closed_date', 'community_board',
       'complaint_type', 'created_date', 'cross_street_1', 'cross_street_2',
       'descriptor', 'due_date', 'facility_type', 'incident_address',
       'incident_zip', 'intersection_street_1', 'intersection_street_2',
       'landmark', 'latitude', 'location', 'location_type', 'longitude',
       'open_data_channel_type', 'park_borough', 'park_facility_name',
       'resolution_action_updated_date', 'resolution_description', 'road_ramp',
       'status', 'street_name', 'taxi_pick_up_location', 'unique_key',
       'x_coordinate_state_plane', 'y_coordinate_state_plane'],
      dtype='object')

In [35]:
data.complaint_type.value_counts()

HEAT/HOT WATER                         146
Illegal Parking                        111
Noise - Residential                     95
Rodent                                  95
Blocked Driveway                        81
Noise                                   77
Street Condition                        40
Noise - Street/Sidewalk                 27
Noise - Vehicle                         19
Noise - Commercial                      18
Sewer                                   15
Sanitation Condition                    14
Water System                            13
General Construction/Plumbing           13
Request Large Bulky Item Collection     13
Derelict Vehicle                        13
UNSANITARY CONDITION                    12
Construction Safety Enforcement         11
Unsanitary Animal Pvt Property          11
PAINT/PLASTER                           10
Dirty Conditions                         9
Food Poisoning                           9
Street Light Condition                   8
PLUMBING   

In [36]:
data.borough.value_counts()

BROOKLYN         307
MANHATTAN        237
QUEENS           225
BRONX            180
STATEN ISLAND     47
Unspecified        4
Name: borough, dtype: int64

In [19]:
test = data.groupby(['borough', 'complaint_type'])

In [37]:
data['complaint_type'][data['borough']=='BROOKLYN'].value_counts()

Illegal Parking                        45
Rodent                                 39
HEAT/HOT WATER                         33
Noise - Residential                    29
Blocked Driveway                       24
Street Condition                       13
Noise                                  13
Noise - Commercial                      8
General Construction/Plumbing           7
Water System                            6
PAINT/PLASTER                           6
Sanitation Condition                    5
Sewer                                   4
Noise - Helicopter                      4
Derelict Vehicle                        4
Unsanitary Animal Pvt Property          4
Derelict Vehicles                       4
UNSANITARY CONDITION                    4
FLOORING/STAIRS                         4
Noise - Street/Sidewalk                 4
Noise - Vehicle                         3
PLUMBING                                3
Air Quality                             3
Dirty Conditions                  

In [38]:
data['complaint_type'][data['borough']=='QUEENS'].value_counts()


Blocked Driveway                       35
Illegal Parking                        34
Rodent                                 16
Noise - Residential                    15
HEAT/HOT WATER                         14
Noise                                  14
Request Large Bulky Item Collection    11
Street Condition                        8
Sanitation Condition                    6
Derelict Vehicle                        6
Noise - Vehicle                         6
Noise - Street/Sidewalk                 5
Building/Use                            4
Food Poisoning                          4
Noise - Commercial                      4
General Construction/Plumbing           4
Derelict Vehicles                       3
Street Sign - Missing                   3
Graffiti                                3
ELECTRIC                                2
Water System                            2
Elevator                                2
Air Quality                             2
Unsanitary Animal Pvt Property    

In [39]:
data['complaint_type'][data['borough']=='MANHATTAN'].value_counts()


Noise                                  46
HEAT/HOT WATER                         41
Noise - Residential                    22
Rodent                                 18
Noise - Street/Sidewalk                15
Construction Safety Enforcement        11
Illegal Parking                         8
Street Condition                        7
Noise - Vehicle                         7
Noise - Commercial                      6
Sewer                                   5
PLUMBING                                4
Dirty Conditions                        3
UNSANITARY CONDITION                    3
PAINT/PLASTER                           3
Food Poisoning                          3
Homeless Person Assistance              3
For Hire Vehicle Complaint              2
Water System                            2
DOOR/WINDOW                             2
Unsanitary Animal Pvt Property          2
WATER LEAK                              2
Blocked Driveway                        2
GENERAL                           

In [42]:
data['complaint_type'][data['borough']=='BRONX'].value_counts()


HEAT/HOT WATER                    58
Noise - Residential               27
Rodent                            20
Blocked Driveway                  15
Illegal Parking                   14
Street Condition                   6
UNSANITARY CONDITION               4
Sewer                              3
Noise                              3
Elevator                           3
Noise - Street/Sidewalk            3
Sanitation Condition               3
Street Light Condition             2
Dirty Conditions                   2
Derelict Vehicle                   2
Unsanitary Animal Pvt Property     2
PLUMBING                           1
Standing Water                     1
GENERAL                            1
Animal Abuse                       1
Noise - Vehicle                    1
Sidewalk Condition                 1
Hazardous Materials                1
Food Poisoning                     1
ELECTRIC                           1
Traffic                            1
New Tree Request                   1
W

In [41]:
data['complaint_type'][data['borough']=='STATEN ISLAND'].value_counts()


Illegal Parking                      10
Street Condition                      6
Blocked Driveway                      5
Water System                          3
Standing Water                        2
Sewer                                 2
Rodent                                2
WATER LEAK                            2
Noise - Vehicle                       2
Noise - Residential                   2
Electronics Waste Appointment         2
Street Light Condition                2
Litter Basket / Request               1
Unsanitary Animal Pvt Property        1
Traffic Signal Condition              1
UNSANITARY CONDITION                  1
Noise                                 1
Derelict Vehicle                      1
Missed Collection (All Materials)     1
Name: complaint_type, dtype: int64